# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# check current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initialize empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

## Create Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

## Set Keyspace

In [12]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create table music_library

In [14]:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(sessionId int, itemInSession int, artist text,firstName text, gender text, lastName text, song_length float, level text, location text, song_title text, userid int, PRIMARY KEY (sessionid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

## Load the data from event_datafile_new.csv

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (sessionid, itemInSession, artist,firstName,gender,lastName,song_length,level,location,song_title,userid)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        artist,firstName,gender,itemInSession,lastName,song_length,level,location,sessionId,song_title,userid = line
        session.execute(query,(int(sessionId),int(itemInSession),artist,firstName,gender,lastName,float(song_length),level,location,song_title,int(userid)))

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## Create music_app_history with the attributes: sessionid, iteminsession, artist, song_title, song_length
## Primary cluster columns will be sessionid and iteminsession

In [17]:
session.execute("""
    CREATE TABLE IF NOT EXISTS music_app_history
    (sessionid int, iteminsession int, artist text,song_title text, song_length float,  PRIMARY KEY (sessionid, itemInSession))
    """) 

## Load data for music_app_history from the file

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        artist,firstName,gender,itemInSession,lastName,song_length,level,location,sessionId,song_title,userid = line
        session.execute(query,(int(sessionId),int(itemInSession), artist,song_title,float(song_length)))

## Load validation for music_app_history table

In [19]:
rows = session.execute("""SELECT artist,song_title, song_length FROM music_app_history where sessionid = 338 and iteminsession = 4""")

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2: Output attributes: artist name, song title (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
## Create  user with the attributes: sessionid, iteminsession, userid, artist, song_title, firstname, lastname and filter based on sessionid and userid.
## We will create primary cluster columns on sessionid, userid, iteminsession

In [25]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_session
    (sessionid int, userid int, iteminsession int, artist text,song_title text, firstname text, lastname text, PRIMARY KEY ((sessionid, userid), iteminsession))
    """) 

## Load the data from the file event_datafile_new.csv

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId,userid,itemInSession,artist,song_title,firstname, lastname)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        artist,firstname,gender,iteminsession,lastname,song_length,level,location,sessionId,song_title,userid = line
        session.execute(query,(int(sessionId),int(userid),int(itemInSession),artist,song_title,firstname,lastname))

## Validate the records for query 2
## Output attributes: artist name, song title (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

In [27]:
rows = session.execute("""SELECT artist, song_title, firstname, lastname FROM user_session WHERE sessionid = 182 and userid = 10""")

for row in rows:
    print(row.artist, row.song_title, row.firstname, row.lastname)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3: List every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## Create a table  with the attributes: song_title, userid, firstname, lastname and filter based on song_title.
## We will create primary cluster columns on song_title, userid                   

In [29]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_songs
    (song_title text, userid int, firstname text, lastname text, PRIMARY KEY (song_title, userid))
    """) 

## Load the data

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (song_title, userid, firstname, lastname)"
        query = query + " VALUES (%s,%s,%s,%s)"
        artist,firstname,gender,iteminsession,lastname,song_length,level,location,sessionId,song_title,userid = line
        session.execute(query,(song_title,int(userid),firstname,lastname))

## Validate the data

In [31]:
rows = session.execute("""SELECT firstname, lastname FROM user_songs WHERE song_title = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [20]:
## TO-DO: Drop the table before closing out the sessions

In [21]:
session.execute("""DROP TABLE music_library""")

In [32]:
session.execute("""DROP TABLE music_app_history""")

In [33]:
session.execute("""DROP TABLE user_session""")

In [34]:
session.execute("""DROP TABLE user_songs""")

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()